# Notebook Instructions

1. If you are new to Jupyter notebooks, please go through this introductory manual <a href='https://quantra.quantinsti.com/quantra-notebook' target="_blank">here</a>.
1. Any changes made in this notebook would be lost after you close the browser window. **You can download the notebook to save your work on your PC.**
1. Before running this notebook on your local PC:<br>
i.  You need to set up a Python environment and the relevant packages on your local PC. To do so, go through the section on "**Run Codes Locally on Your Machine**" in the course.<br>
ii. You need to **download the zip file available in the last unit** of this course. The zip file contains the data files and/or python modules that might be required to run this notebook.

# Initialise and Reset Game Class

In the previous units, you learned the idea of gamification, where we treat each trade as an individual game. To play this trading game, we create a class called `Game`. In the Game class, we will generate input features and assemble states. These states will pass to the agent or neural networks which will predict the action which is buy, sell or hold. We will also update the position of the trade and calculate the reward when the game is over. You will learn all these things one by one in a stepwise manner in the later sections of this course. But before doing all that, we need to initialise the Game class. 

In this notebook, you will perform the following steps:
1. [Read price data](#read)
2. [Resample price data ](#resample)
3. [Initialise Game class](#game)

<a id='read'></a> 
## Read Price data

The OHLCV data is stored in the compressed pickle file. This is the 5 minutes data starting from 2010-04-01 to 2020-08-31. You can download this data from the last section of this course '**Python Codes and Data**' unit.

To read a pickle file, you can use `read_pickle` method of pandas. The syntax is shown below.

Syntax: 
```python
import pandas as pd
pd.read_pickle(filename)
```
filename: name of the file in the string format. The extension of the compressed file is bz2.

In [1]:
# Import pickle
import pandas as pd

# Import datetime
from datetime import datetime, timedelta
import datetime

# The data is stored in the directory 'data_modules'
path = '../data_modules/'

# Read the pickle file
bars5m = pd.read_pickle(path+ 'PriceData5m.bz2')
bars5m.head()

,open,high,low,close,volume
Time,,,,,
2010-01-04 09:35:00-05:00,91.711,91.809,91.703,91.760,4448908.0
2010-01-04 09:40:00-05:00,91.752,91.973,91.752,91.932,4380988.0
2010-01-04 09:45:00-05:00,91.940,92.022,91.928,92.005,2876633.0
2010-01-04 09:50:00-05:00,92.005,92.177,91.973,92.177,4357079.0
2010-01-04 09:55:00-05:00,92.168,92.177,92.038,92.079,2955068.0


<a id='resample'></a> 
## Resample price data

We use `resample()` and `agg()` method for the resampling of time series. 

Syntax: 
```python
DataFrame.resample(frequency, label, closed).agg(func)
```

Parameters:

    frequency: Frequency of resampling. Some of the values are '1D' to resample to the daily frequency and '1H' to resample to hourly frequency.
    
    label: Label parameter takes 'left', 'right' as an input. The 'label' parameter is used to choose whether start or end are used as a representative of the interval. For example, the original 5 mins data starts at 2010-01-04 09:35:00 and ends at 2020-08-31 16:00:00. If you want to resample the data to 1 hour and choose left as an input, it will keep the left side of the data that is it starts at 2010-01-04 09:30:00 and ends at 2020-08-31 15:00:00. If you want to choose right as an input, it will keep the right side of the data that is it starts at 2010-01-04 10:00:00 and ends at 2020-08-31 16:00:00. 
    
    closed: Closed parameter takes 'left', 'right' as an input. The 'closed' parameter is used to set the strict vs non-strict inequality to perform the action (in our case it's to aggregate using one or more operations). 
    
    left: (start, end]
    right: [start, end)
   
For more details on closed and label parameter, refer this <a href="https://stackoverflow.com/questions/48340463/how-to-understand-closed-and-label-arguments-in-pandas-resample-method" target="_blank"> link</a>.  

    func: Function to use for aggregating the data. For a dataframe, can pass a dictionary, if the keys are dataframe column names.

Returns: DataFrame with resampled time series

In [2]:
# Create a dictionary to map the open, high, low, close, volume
# open is the 'first' value in the set of defined frequency
# high is the 'max' value in the set of defined frequency 
# low is the min value in the set of defined frequency 
# close is the 'last' value in the set of defined frequency
# volume is the 'sum' of volume in the set of defined frequency

ohlcv_dict = {
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }

In [3]:
bars1h = bars5m.resample('1H', label='left', closed='right').agg(ohlcv_dict).dropna()
bars1h.head()

,open,high,low,close,volume
Time,,,,,
2010-01-04 09:00:00-05:00,91.711,92.177,91.703,92.091,20908095.0
2010-01-04 10:00:00-05:00,92.095,92.422,92.005,92.307,22280055.0
2010-01-04 11:00:00-05:00,92.307,92.544,92.307,92.450,11781384.0
2010-01-04 12:00:00-05:00,92.454,92.519,92.430,92.466,6703479.0
2010-01-04 13:00:00-05:00,92.495,92.536,92.364,92.381,7132690.0


In [4]:
bars1d = bars1h.resample('1D', label='left', closed='right').agg(ohlcv_dict).dropna()
bars1d.head()

,open,high,low,close,volume
Time,,,,,
2010-01-04 00:00:00-05:00,91.711,92.544,91.703,92.487,95717178.0
2010-01-05 00:00:00-05:00,92.438,92.781,92.103,92.740,96560334.0
2010-01-06 00:00:00-05:00,92.650,93.034,92.577,92.830,96116363.0
2010-01-07 00:00:00-05:00,92.634,93.311,92.373,93.205,106214432.0
2010-01-08 00:00:00-05:00,92.952,93.540,92.764,93.507,96925188.0


<a id='game'></a> 
## Initialise Game class

We have discussed the idea of gamification where we treat each individual trade as a game with start, play period, and end. In our RL algorithm, this is handled by the `Game` class. When a new trade is initiated, we create a new instance of the Game class. And reset all its state values to default by calling the `self.reset()` function, when the individual trade game is over. 

In [5]:
class Game(object):

    def __init__(self, bars5m, bars1d, bars1h, reward_function, lkbk=20,  init_idx=None):

        # Initialise 5 mins frequency data
        self.bars5m = bars5m
        # Initilaise lookback period for the calculation of technical indicators
        self.lkbk = lkbk
        # Intialise length of each trade
        self.trade_len = 0
        # Initialise 1 day frequency data
        self.bars1d = bars1d
        # Initialise 1 hour frequency data
        self.bars1h = bars1h
        # Initialise when game is over to update the state, position and calculate reward
        self.is_over = False
        # Intialise reward to store the value of reward
        self.reward = 0
        # Define pnl_sum to calculate the pnl when all episodes are complete.
        self.pnl_sum = 0
        # Supply a starting index which indicates a position in our price dataframe
        # and denotes the point at which the game starts
        self.init_idx = init_idx
        # Instantiate reward function
        self.reward_function = reward_function
        # When game is over, reset all state values
        self.reset()

    def reset(self):
        """When an individual trade game is over, we reset the system for new trading game.
        """
        self.pnl = 0
        self.entry = 0
        self._time_of_day = 0
        self._day_of_week = 0
        self.curr_idx = self.init_idx
        self.t_in_secs = (
            self.bars5m.index[-1]-self.bars5m.index[0]).total_seconds()
        self.start_idx = self.curr_idx
        self.curr_time = self.bars5m.index[self.curr_idx]
        self._get_last_N_timebars()
        self.position = 0  
        self.act(0)
        self.state = []
        self._assemble_state()

The whole trading game is played inside the Game class. In this notebook you learned to initialise the Game class. In the later sections, you will learn to assemble the state, update the trading position and calculate reward in the individual notebooks and finally fit all these inside the Game class to complete that.<br><br>